# Make a cv loop that returns the average training and cv error and bias
## Requirments
### Easily incorporated into my tunnel organising the T data


#### The  function takes as input the 
data with an 'actual sales' column and a 'forecast sales' column

#### it returns, for training and test data
print outs of bias and error   
my tracker with bias and error info appended

#### My 'evaluate' function takes as input the 
model   
training/test data (with all features)  
column names for features used in the model   
target data   

#### it returns, for training and test data
print outs of bias and error   
my tracker with bias and error info appended

In [21]:
import pandas as pd
import numpy as np

In [13]:
evaluation_table = pd.DataFrame(columns=['Model','Train error', 'CV error', 'CV bias'])

In [26]:
def cv_model(model, model_name, data, features, k, df = evaluation_table
             find_train_error = True,
             find_best_error = True, find_bias = True):
    '''Returns df appended with the average training error and k-fold cross-validation error of model. 
    Only the columns in features are used to train model.
    Prints performance metrics relating to model's performance on the training data and during cross-validation.'''
    
    # Initalise sums for bias, error, best_error for training, validation
    sum_bias_train = 0
    sum_error_train = 0
    sum_best_train = 0

    sum_bias_cv = 0
    sum_error_cv = 0
    sum_best_cv = 0
    
    # Count number of data points
    n = data.shape[0]

    # For each cross-validation set
    for i in range(k):    
        ## Define training and validation sets
        train = data[0:n*i//k].copy()
        train = train.append(data[n*(i+1)//k:])
        val = data[n*i//k:n*(i+1)//k].copy()
        ## Train the model on the training set
        model.fit(train[features], train['adjusted_sales_singles'])

        ## Calculate bias, error on the training set
        if find_train_error:
            train['forecast_sales'] = model.predict(train[features])
            data_modified = aggregate_slow_sellers(data_modified)

            metrics = forecast_score(train, find_best_error, find_bias)
            ## Add to training sums
            sum_bias_train += metrics[1]
            sum_error_train += metrics[2]
            sum_best_train += metrics[3]

        ## Calculate bias, error on the validation set
        val['forecast_sales'] = model.predict(val[features])
        metrics = forecast_score(val, find_best_error, find_bias)
        ## Add to cv sum
        sum_bias_cv += metrics[1]
        sum_error_cv += metrics[2]
        sum_best_cv += metrics[3]

    # Calculate average bias, error on training, validation
    av_bias_train = sum_bias_train/k
    av_error_train = sum_error_train/k
    av_best_train = sum_best_train/k

    av_bias_cv = sum_bias_cv/k
    av_error_cv = sum_error_cv/k
    av_best_cv = sum_best_cv/k    
    
    # Print results for training, validation
    if find_train_error:
        print ('Performance on training data:')
        print ('Lowest possible error: ', av_best_train)
        print ('Error: ', av_error_train)
        print ('Bias: ', av_error_train, '\n')

    print ('Performance during cross-validation:')
    print ('Lowest possible error: ', av_best_cv)
    print ('Error: ', av_error_cv)
    print ('Bias: ', av_error_cv)    
    
    # Return tracker appended with bias and error info
    if not find_train_error:
        av_error_train = np.NaN
    return df.append({'Model': model_name, 
                                    'Train error': av_error_train,
                                    'CV error': av_error_cv,
                                    'CV bias': av_bias_cv}, ignore_index = True)

SyntaxError: invalid syntax (<ipython-input-26-e0f4d3e0e467>, line 2)

In [16]:
def aggregate_slow_sellers(data_frame, cols={'actual_sales': 'adjusted_sales_singles', 
                                             'forecast_sales': 'forecast_sales', 
                                             'base_product_number': 'base_product_number', 
                                             'calendar_date': 'calendar_date', 
                                             'retail_outlet_number': 'retail_outlet_number'}):
    """Aggregate the forecast for days with zero sales
    
    Args:
        data_frame - pandas DataFrame with actual and forecast sales.
        cols - dictionary of column name mapping:
            * cols['actual_sales'] = the column containing actual sales for the store, day, product
            * cols['forecast_sales'] = the column containing forecast sales for the store, day, product
            * cols['base_product_number'] = the column containing the product number
            * cols['calendar_date'] = the column contianing the date
            * cols['retail_outlet_number'] = the column containing the retail outlet number
    
    Return
        data_frame - with the forecast for days with zero sales carried forward
    """
    df_work = data_frame.copy()
    df_work = df_work.sort_values([cols['base_product_number'], 
                                   cols['retail_outlet_number'], 
                                   cols['calendar_date']]).reset_index(drop=True)
    
    df_work['carry_over'] = 0.0
    
    # iterate
    last_store = df_work.at[0, cols['retail_outlet_number']]
    last_product = df_work.at[0, cols['base_product_number']]
    carry_over = 0.0
    
    for i in range(len(df_work)):
        current_store = df_work.at[i, cols['retail_outlet_number']]
        current_product = df_work.at[i, cols['base_product_number']]
        current_sales = df_work.at[i, cols['actual_sales']]
        current_forecast = df_work.at[i, cols['forecast_sales']]
        
        # different product/store so reset
        if current_store != last_store or current_product != last_product:
            carry_over = 0.0
        
        carry_over += current_forecast
        df_work.at[i, 'carry_over'] = carry_over

        # if we sold something reset
        if current_sales > 0.0:
            carry_over = 0.0
            
        last_store = current_store
        last_product = current_product
    
    # the forecast sales are now the carry over
    df_work[cols['forecast_sales']] = df_work['carry_over']
    # drop carry_over column and days with no sales
    df_work = df_work.drop('carry_over', axis=1)
    df_work = df_work[df_work[cols['actual_sales']] != 0]
    
    return df_work

In [19]:
def forecast_score(data_frame, find_best_error = True, find_bias = True,
                   cols={'actual_sales': 'adjusted_sales_singles', 
                                     'forecast_sales': 'forecast_sales'}):
    """
    Args
        data_frame - pandas DataFrame with actual and forecast sales
        cols - dictionary of column name mapping:
            * cols['actual_sales'] = the column containing actual sales for the store, day, product
            * cols['forecast_sales'] = the column containing forecast sales for the store, day, product
    Return
        Total sales, bias, forecast error, Best Error
    """
    
    total_sales = data_frame[cols['actual_sales']].sum()
    
    
    forecast_error = np.sum((np.abs(data_frame[cols['forecast_sales']] 
                                    - data_frame[cols['actual_sales']]))) / total_sales
        
    if find_bias:
        bias =  np.sum((data_frame[cols['actual_sales']] 
                    - data_frame[cols['forecast_sales']])) / total_sales
    else:
        bias = 'Bias not calculated'
    
    if find_best_error:
        best_error = np.sum(0.8 * np.sqrt(data_frame[cols['actual_sales']])) / total_sales
    else:
        best_error = 'Best error not calculated'
    
    return total_sales, bias, forecast_error, best_error

In [22]:
np.NaN

nan

In [24]:
evaluation_table.append({'Model': np.NaN, 
                                    'Train error': np.NaN,
                                    'CV error': np.NaN,
                                    'CV bias': np.NaN}, ignore_index = True)

,Model,Train error,CV error,CV bias
0,NaN,NaN,NaN,NaN


In [27]:
def f(x=1, y=1, z=1):
    return x*y*z

In [29]:
f(5)

5

In [31]:
f(7,2,2)

28